In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
import genesis as gs
# show the iamge
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from genesis_gym import GenesisGym

[I 05/02/25 16:56:28.267 386026] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [3]:
import random
import pathlib as pl
import numpy as np
import os, sys
import torch

## Default Args
DEFAULT_RADIUS = 0.034
DEFAULT_HEIGHT = 0.09
DEFAULT_RHO = 2000
DEFAULT_FRICTION = 0.5
DEFAULT_STARTING_X = 0.65
    # parser = argparse.ArgumentParser(description='Genesis Gym Environment')
    # parser.add_argument('--vis', action='store_true', help='Enable visualization')
    # parser.add_argument('--radius', type=float, default=DEFAULT_RADIUS, help='Bottle radius')
    # parser.add_argument('-e', '--height', type=float, default=DEFAULT_HEIGHT, help='Bottle height')
    # parser.add_argument('-o', '--rho', type=float, default=DEFAULT_RHO, help='Density of the bottle')
    # parser.add_argument('--friction', type=float, default=DEFAULT_FRICTION, help='Friction of the bottle')
    # parser.add_argument('--starting_x', type=float, default=DEFAULT_STARTING_X, help='Starting x position of the bottle')
    # parser.add_argument('--max-demos', type=int, default=1e7, help='Max number of demos to load')
    # parser.add_argument('--random-agent', action='store_true', help='Use a random agent')
    # parser.add_argument('--subsample', type=int, default=2, help='Subsample ratio for the demos')
    # parser.add_argument('--env-name', type=str, default='lift', help='Environment name')
    # args = parser.parse_args()

args = {
    'vis': True,
    'radius': DEFAULT_RADIUS,
    'height': DEFAULT_HEIGHT,
    'rho': DEFAULT_RHO,
    'friction': DEFAULT_FRICTION,
    'starting_x': DEFAULT_STARTING_X,
    'max_demos': 1e7,
    'random_agent': False,
    'subsample': 2,
    'env_name': 'lift',
}

env = GenesisGym(**args)
obs = env.reset()

GenesisGym Task=lift args: {'rho': 2000, 'radius': 0.034, 'height': 0.09, 'friction': 0.5, 'vis': True, 'grayscale': False, 'time_limit': 4000, 'env_name': 'lift'}
Joint 'right_finger_tip_joint' mimics 'right_finger_bottom_joint' with multiplier -0.676 and offset -0.05
Joint 'left_finger_bottom_joint' mimics 'right_finger_bottom_joint' with multiplier -1.0 and offset 0.0
Joint 'left_finger_tip_joint' mimics 'right_finger_bottom_joint' with multiplier -0.676 and offset -0.05


[W 05/02/25 16:56:37.291 386026] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


In [ ]:
from demo_holder import GenesisDemoHolder
from collections import defaultdict
demo_player = GenesisDemoHolder(use_eef=False, subsample_ratio=args['subsample'])
TRIAL_CAN_ADJUSTED = defaultdict(lambda: False)


Loaded 96 demos from /home/j/workspace/genesis_sim2real/inthewild_trials
234 (450, 7) -- 246 (1311, 7) -- 331 (1434, 7) -- 236 (339, 7) -- 307 (943, 7) -- 326 (483, 7) -- 263 (525, 7) -- 278 (334, 7) -- 310 (917, 7) -- 257 (504, 7) -- 260 (880, 7) -- 284 (657, 7) -- 239 (1378, 7) -- 295 (3116, 7) -- 248 (1531, 7) -- 277 (427, 7) -- 261 (706, 7) -- 305 (631, 7) -- 297 (569, 7) -- 322 (211, 7) -- 303 (1152, 7) -- 269 (2428, 7) -- 283 (660, 7) -- 318 (759, 7) -- 324 (324, 7) -- 321 (857, 7) -- 333 (2035, 7) -- 243 (286, 7) -- 273 (734, 7) -- 270 (1850, 7) -- 280 (362, 7) -- 237 (401, 7) -- 267 (886, 7) -- 301 (826, 7) -- 309 (877, 7) -- 259 (604, 7) -- 245 (1662, 7) -- 289 (301, 7) -- 253 (597, 7) -- 244 (524, 7) -- 300 (2385, 7) -- 252 (771, 7) -- 247 (868, 7) -- 268 (2538, 7) -- 265 (1464, 7) -- 328 (1383, 7) -- 279 (1000, 7) -- 302 (582, 7) -- 285 (418, 7) -- 288 (1664, 7) -- 250 (1675, 7) -- 266 (895, 7) -- 233 (463, 7) -- 314 (650, 7) -- 315 (1945, 7) -- 296 (1395, 7) -- 294 (2682, 7

In [ ]:

demo_player.next_demo()
trial_id = demo_player.get_trial_id()
env.reset(trial_id=trial_id)
action = demo_player.next_action()['action']
while action is not None:
    obs, reward, done, info = env.step(action)
    if args['vis']:
        env.render()
    action = demo_player.next_action()
    action = action['action'] if action is not None else None
    if done:
        break


    gripper_pos = env.kinova.get_link('end_effector_link').get_pos().cpu().numpy()
    can_pose = env.bottle.get_pos().cpu().numpy()
    dp = np.linalg.norm(gripper_pos - can_pose)
    if action[-1] > 0.5 and dp < 0.2 and not TRIAL_CAN_ADJUSTED[trial_id] and gripper_pos[2] < 0.5:
        # get the average pos of the last 4 links 
        grip_pos = env.get_grip_pose() + [0.05, 0, 0]
        env.reset(trial_id=trial_id)
        demo_player.reset_current_demo()
        env.set_can_to_pose(torch.Tensor(grip_pos))
        print("Gripper closing and can is nearby, restarting demo and setting can to gripper pose")
        TRIAL_CAN_ADJUSTED[trial_id] = True